In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
from scipy.io import loadmat
# Replace 'your_file.mat' with the path to your actual .mat file
mat = loadmat('/content/gdrive/MyDrive/inputdata.mat')
# mat is now a dictionary with variable names as keys and loaded matrices as values
# To access a variable named 'VariableName' stored in the .mat file, use:
X = mat['inputdata']
# Now, data contains the array stored in 'VariableName'. You can manipulate it as needed.

In [ ]:
from scipy.io import loadmat
# Replace 'your_file.mat' with the path to your actual .mat file
mat = loadmat('/content/gdrive/MyDrive/labeldata.mat')
# mat is now a dictionary with variable names as keys and loaded matrices as values
# To access a variable named 'VariableName' stored in the .mat file, use:
y = mat['labeldata']
# Now, data contains the array stored in 'VariableName'. You can manipulate it as needed.

In [ ]:
# Import required libraries
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, LSTM, Input, MultiHeadAttention, LayerNormalization, concatenate, GlobalAveragePooling1D, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_curve, auc, cohen_kappa_score
import numpy as np
import matplotlib.pyplot as plt

# Assuming X and y are preloaded datasets
# Ensure y is processed for categorical classification
y_categorical = to_categorical(y) if y.ndim == 1 else y

# Split the data into train, validation, and test sets
X_temp, X_test, y_temp, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

# Reshape X matrices to 3D if they are 2D, for Conv1D layers
for dataset in [X_train, X_test, X_val]:
    if dataset.ndim == 2:
        dataset = dataset.reshape(dataset.shape[0], dataset.shape[1], 1)

# Define the transformer encoder component
def transformer_encoder(inputs, num_heads=4, ff_dim=64, sequence_length=18):
    attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=ff_dim)(inputs, inputs)
    attention_output = Dropout(0.1)(attention_output)
    out1 = LayerNormalization(epsilon=1e-6)(inputs + attention_output)
    ff_output = Dense(ff_dim, activation="relu")(out1)
    ff_output = Dropout(0.1)(ff_output)
    ff_output = Dense(sequence_length, activation="relu")(ff_output)
    return LayerNormalization(epsilon=1e-6)(out1 + ff_output)

# Model creation
def create_model(sequence_length=18, num_classes=2):
    inputs = Input(shape=(sequence_length, 1))
    x1 = Conv1D(filters=32, kernel_size=3, activation='relu')(inputs)
    x1 = BatchNormalization()(x1)
    x1 = MaxPooling1D(pool_size=2)(x1)
    x1 = LSTM(64, return_sequences=False)(x1)
    x2 = transformer_encoder(inputs)
    x2 = GlobalAveragePooling1D()(x2)
    merged = concatenate([x1, x2])
    x = Dense(128, activation='relu')(merged)
    x = Dropout(0.2)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Initialize and compile the model
model = create_model(sequence_length=X_train.shape[1], num_classes=y_categorical.shape[1])
model.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, mode='min', verbose=1),
    ModelCheckpoint(filepath='best_hybrid_model.h5', monitor='val_loss', mode='min', save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-5, verbose=1)
]

# Train the model
history = model.fit(X_train, y_train, batch_size=64, epochs=30, validation_data=(X_val, y_val), callbacks=callbacks)

# Load the best model
model = load_model('best_hybrid_model.h5')

# Predict and evaluate
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred))

# Plot accuracy and loss
plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Val')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Val')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

# Plot ROC curve
fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob[:, 1])
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

# Calculate and print Kappa Score
kappa_value = cohen_kappa_score(y_true, y_pred)
print(f'Kappa Score: {kappa_value}')